In [5]:
import numpy as np
import copy as cp
import random

def add_score(sc, val):
    sc += val
    return sc

def move_left(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[i, :] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i, :] = np.array(non_zero + zero)
        for j in range(3):
            if grid[i, j] == grid[i, j+1]:
                grid[i, j] *= 2
                score = add_score(score, grid[i, j])
                grid[i, j+1] = 0
        non_zero = [x for x in grid[i, :] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i, :] = np.array(non_zero + zero)
    return grid, score

def move_right(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[i, :] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i, :] = np.array(zero + non_zero[::-1])
        for j in range(3, 0, -1):
            if grid[i, j] == grid[i, j-1]:
                grid[i, j] *= 2
                score = add_score(score, grid[i, j])
                grid[i, j-1] = 0
        non_zero = [x for x in grid[i, :] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[i, :] = np.array(zero + non_zero[::-1])
    return grid, score

def move_up(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[:, i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:, i] = np.array(non_zero + zero)
        for j in range(3):
            if grid[j, i] == grid[j+1, i]:
                grid[j, i] *= 2
                score = add_score(score, grid[j, i])
                grid[j+1, i] = 0
        non_zero = [x for x in grid[:, i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:, i] = np.array(non_zero + zero)
    return grid, score

def move_down(grid, score):
    for i in range(4):
        non_zero = [x for x in grid[:, i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:, i] = np.array(zero + non_zero[::-1])
        for j in range(3, 0, -1):
            if grid[j, i] == grid[j-1, i]:
                grid[j, i] *= 2
                score = add_score(score, grid[j, i])
                grid[j-1, i] = 0
        non_zero = [x for x in grid[:, i] if x != 0]
        zero = [0] * (4 - len(non_zero))
        grid[:, i] = np.array(zero + non_zero[::-1])
    return grid, score

def add_new_number(grid):
    zero_indices = np.where(grid == 0)
    if len(zero_indices[0]) == 0: return False
    index = np.random.choice(len(zero_indices[0]))
    i, j = zero_indices[0][index], zero_indices[1][index]
    grid[i, j] = 2 if np.random.random() < 0.9 else 4
    return True

def check_game_over(grid):
    if np.all(grid) == False: return False
    for row in range(4):
        for col in range(4):
            if row != 3:
                if grid[row, col] == grid[row+1, col]: return False
            if col != 3:
                if grid[row, col] == grid[row, col+1]: return False
    return True

def check_win(grid): return 2048 in grid

def play_2048(grid, move, score):
    orig_grid = cp.deepcopy(grid)
    if check_game_over(grid): raise RuntimeError("GO")
    if move == 'left': grid, score = move_left(grid, score)
    elif move == 'right': grid, score = move_right(grid, score)
    elif move == 'up': grid, score = move_up(grid, score)
    elif move == 'down': grid, score = move_down(grid, score)
    else: raise ValueError("Invalid move")
    if check_win(grid): raise RuntimeError("WIN")
    if not np.array_equal(grid, orig_grid): add_new_number(grid)
    return grid, score

def new_game():
    score = 0
    grid = np.zeros((4,4), dtype=int)
    add_new_number(grid)
    add_new_number(grid)
    return grid, score

def monte_carlo_rollout(grid, score, max_rollout_moves=100):
    grid_copy = cp.deepcopy(grid)
    score_copy = score
    moves_made = 0
    while moves_made < max_rollout_moves:
        move_choice = np.random.choice(['left', 'right', 'up', 'down'])
        try:
            grid_copy, score_copy = play_2048(grid_copy, move_choice, score_copy)
            moves_made += 1
        except RuntimeError as e:
            if str(e) == "WIN": return score_copy + 100000
            else: return score_copy
    return score_copy

def solver_monte_carlo(grid, score, n_simulations=20):
    moves = ['left', 'right', 'up', 'down']
    best_move = None
    best_avg_score = -np.inf
    for move in moves:
        grid_copy = cp.deepcopy(grid)
        score_copy = score
        if move == 'left': new_grid, new_score = move_left(grid_copy, score_copy)
        elif move == 'right': new_grid, new_score = move_right(grid_copy, score_copy)
        elif move == 'up': new_grid, new_score = move_up(grid_copy, score_copy)
        elif move == 'down': new_grid, new_score = move_down(grid_copy, score_copy)
        if np.array_equal(new_grid, grid): continue
        sim_scores = []
        for _ in range(n_simulations):
            grid_sim = cp.deepcopy(new_grid)
            add_new_number(grid_sim)
            sim_score = monte_carlo_rollout(grid_sim, new_score, max_rollout_moves=100)
            sim_scores.append(sim_score)
        avg_score = np.mean(sim_scores) if sim_scores else -np.inf
        if avg_score > best_avg_score:
            best_avg_score = avg_score
            best_move = move
    if best_move is None: best_move = np.random.choice(moves)
    return best_move

def solver_random(grid, score):
    moves = ['left', 'right', 'up', 'down']
    winning_moves = []
    for move in moves:
        grid_copy = cp.deepcopy(grid)
        score_copy = score
        if move == 'left': new_grid, new_score = move_left(grid_copy, score_copy)
        elif move == 'right': new_grid, new_score = move_right(grid_copy, score_copy)
        elif move == 'up': new_grid, new_score = move_up(grid_copy, score_copy)
        elif move == 'down': new_grid, new_score = move_down(grid_copy, score_copy)
        if check_win(new_grid): winning_moves.append(move)
    if winning_moves: return np.random.choice(winning_moves)
    return np.random.choice(moves)

def solver_heuristic(grid, score):
    moves = ['left', 'right', 'up', 'down']
    for move in moves:
        grid_copy = cp.deepcopy(grid)
        score_copy = score
        if move == 'left': new_grid, new_score = move_left(grid_copy, score_copy)
        elif move == 'right': new_grid, new_score = move_right(grid_copy, score_copy)
        elif move == 'up': new_grid, new_score = move_up(grid_copy, score_copy)
        elif move == 'down': new_grid, new_score = move_down(grid_copy, score_copy)
        if check_win(new_grid): return move
    best_move = None
    best_free = -1
    for move in moves:
        grid_copy = cp.deepcopy(grid)
        score_copy = score
        if move == 'left': new_grid, new_score = move_left(grid_copy, score_copy)
        elif move == 'right': new_grid, new_score = move_right(grid_copy, score_copy)
        elif move == 'up': new_grid, new_score = move_up(grid_copy, score_copy)
        elif move == 'down': new_grid, new_score = move_down(grid_copy, score_copy)
        if np.array_equal(new_grid, grid): continue
        free_cells = np.count_nonzero(new_grid == 0)
        if free_cells > best_free:
            best_free = free_cells
            best_move = move
    if best_move is None: best_move = np.random.choice(moves)
    return best_move

def run_game(solver):
    grid, score = new_game()
    move_counts = {"left": 0, "right": 0, "up": 0, "down": 0}
    total_moves = 0
    outcome = None
    while True:
        move = solver(grid, score)
        try:
            grid, score = play_2048(grid, move, score)
            move_counts[move] += 1
            total_moves += 1
        except RuntimeError as e:
            move_counts[move] += 1
            total_moves += 1
            if str(e) == "WIN": outcome = "win"
            else: outcome = "loss"
            break
    max_tile = np.max(grid)
    return {"score": score, "total_moves": total_moves, "move_counts": move_counts, "max_tile": max_tile, "outcome": outcome}

def run_experiments(solver, solver_name, num_games=30):
    results = []
    print(f"\nRunning experiments for: {solver_name}")
    for i in range(num_games):
        stats = run_game(solver)
        results.append(stats)
        print(f"{solver_name} - game {i+1}/{num_games} completed: score {stats['score']}, moves {stats['total_moves']}, max tile {stats['max_tile']}, result {stats['outcome']}")
    return results

def summarize_results(results):
    scores = [r["score"] for r in results]
    total_moves = [r["total_moves"] for r in results]
    max_tiles = [r["max_tile"] for r in results]
    outcomes = [r["outcome"] for r in results]
    move_keys = ["left", "right", "up", "down"]
    avg_moves_dir = {key: np.mean([r["move_counts"][key] for r in results]) for key in move_keys}
    summary = {
        "best_score": np.max(scores),
        "worst_score": np.min(scores),
        "average_score": np.mean(scores),
        "total_games": len(results),
        "wins": outcomes.count("win"),
        "losses": outcomes.count("loss"),
        "average_max_tile": np.mean(max_tiles),
        "average_total_moves": np.mean(total_moves),
        "average_moves_per_direction": avg_moves_dir
    }
    return summary

def print_summary(solver_name, summary):
    win_rate = (summary["wins"] / summary["total_games"]) * 100
    print("\n==============================")
    print(f"Statistics for {solver_name}")
    print("==============================")
    print(f"Total games: {summary['total_games']}")
    print(f"Wins: {summary['wins']} | Losses: {summary['losses']} ({win_rate:.2f}% win rate)")
    print(f"Best score: {summary['best_score']} | Worst score: {summary['worst_score']}")
    print(f"Average score: {summary['average_score']:.2f}")
    print(f"Average highest tile: {summary['average_max_tile']:.2f}")
    print(f"Average total moves: {summary['average_total_moves']:.2f}")
    print("Average moves per direction:")
    for direction, avg in summary["average_moves_per_direction"].items(): print(f"  {direction}: {avg:.2f}")
    print("==============================\n")

def main():
    num_games = 30
    solvers = [
        (solver_random, "Random Solver"),
        (solver_heuristic, "Heuristic Solver"),
        (solver_monte_carlo, "Monte Carlo Solver")
    ]
    for solver_func, name in solvers:
        results = run_experiments(solver_func, name, num_games=num_games)
        summary = summarize_results(results)
        print_summary(name, summary)

if __name__ == '__main__': main()


Running experiments for: Random Solver
Random Solver - game 1/30 completed: score 700, moves 114, max tile 64, result loss
Random Solver - game 2/30 completed: score 2308, moves 237, max tile 256, result loss
Random Solver - game 3/30 completed: score 1288, moves 175, max tile 128, result loss
Random Solver - game 4/30 completed: score 2644, moves 256, max tile 256, result loss
Random Solver - game 5/30 completed: score 1372, moves 162, max tile 128, result loss
Random Solver - game 6/30 completed: score 1204, moves 153, max tile 128, result loss
Random Solver - game 7/30 completed: score 1332, moves 154, max tile 128, result loss
Random Solver - game 8/30 completed: score 708, moves 106, max tile 64, result loss
Random Solver - game 9/30 completed: score 584, moves 99, max tile 64, result loss
Random Solver - game 10/30 completed: score 1752, moves 203, max tile 128, result loss
Random Solver - game 11/30 completed: score 2208, moves 223, max tile 256, result loss
Random Solver - gam